In [ ]:
# Necessary Imports
import cv2
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image

In [ ]:
# Path to Kaggle Input
path = "../input/indian-monuments-image-dataset/Indian-monuments/images"
# Walk through the directory and list number of files
for dirpath, dirnames, filenames in os.walk(path):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
train_dir =  path + "/train/"
test_dir = path + "/test/"
train_dir

In [ ]:
# get all the class names
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Rescale the data and create data generator instances
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# Load data in from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224, 3),
                                               batch_size=32,
                                               class_mode='categorical') 

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(224, 224, 3),
                                              batch_size=32,
                                              class_mode='categorical')

In [ ]:
model_10 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(24, activation='softmax')
])

In [ ]:
# model_10.compile(loss='categorical_crossentropy',
#                  optimizer=tf.keras.optimizers.Adam(),
#                  metrics=['accuracy'])

# history_10 = model_10.fit(train_data,
#                           epochs=5,
#                           steps_per_epoch=len(train_data),
#                           validation_data=test_data,
#                           validation_steps=len(test_data))

In [ ]:
# plot_loss_curves(history_10)

In [ ]:
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D


# Load the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32)



# Define the DenseNet model
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Add a global average pooling layer
x = train_data
x = tf.keras.applications.densenet.preprocess_input(x)
x = densenet(x)
x = GlobalAveragePooling2D()(x)

# Add a dense layer
predictions = Dense(24, activation='softmax')

# Define the model
model = Sequential()
model.add(densenet)
model.add(predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit_generator(train_generator, epochs=10, validation_data=validation_generator)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0


# Load the EfficientNetB0 model.
model = tf.keras.applications.EfficientNetB0(weights='imagenet')


# Compile the model.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on your dataset.
model = model_10.fit(train_data,
                          epochs=5,
                          steps_per_epoch=len(train_data),
                          validation_data=test_data,
                          validation_steps=len(test_data))
